**Convolutional Layers**
==

So to do image recognition we need to create a convolutional network!

This takes photo data!


In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

color_channels = 1
image_height = 28
image_width  = 28


class CNN:
    def __init__(self, image_height, image_width, channels, num_classes):
        #this input layer holds the image data as is. in the tensor placeholder 
        self.input_layer = tf.placeholder(dtype=tf.float32, shape=[None, image_height, image_width, channels], name="inputs")
        print(self.input_layer.shape)
        
        #this convolutional layer takes the input data and grabs the kernel_size and breaks
        #that size off and passes it through an activation function that will test similarities to the rest of the image,
        conv_layer_1 = tf.layers.conv2d(self.input_layer, filters=32, kernel_size=[2, 2], padding="same", activation=tf.nn.relu)
        print(conv_layer_1.shape)
         
        #the pooling layer removes the precision of the image data
        pooling_layer_1 = tf.layers.max_pooling2d(conv_layer_1, pool_size=[2,2], strides=2)
        print(pooling_layer_1.shape)
         
        #runs a second convolutional layer to take chunk up a second time
        conv_layer_2 = tf.layers.conv2d(pooling_layer_1, filters=64, kernel_size=[2, 2], padding="same", activation=tf.nn.relu)
        print(conv_layer_2.shape)
        
        #the pooling layer does the same  shrinkning our# data sets
        pooling_layer_2 = tf.layers.max_pooling2d(conv_layer_2, pool_size=[2, 2], strides=2)
        print(pooling_layer_2.shape)
        
        #flatten the data
        flattened_pooling = tf.layers.flatten(pooling_layer_2)
        print("Flattened: " + str(flattened_pooling.shape) )
        
        #develops a neural network containing 1024
        dense_layer = tf.layers.dense(flattened_pooling, 1024, activation=tf.nn.relu)
        print("Dense: " + str(dense_layer.shape))
        dropout = tf.layers.dropout(dense_layer, rate=0.4, training=True)
        print("Dropout: " + str(dropout.shape))
        outputs = tf.layers.dense(dropout, num_classes)
        print("Outputs: " + str(outputs.shape))
         
        self.choice = tf.argmax(outputs, axis=1)
        self.probability = tf.nn.softmax(outputs)
         
        self.labels = tf.placeholder(dtype=tf.float32, name="labels")
        self.accuracy, self.accuracy_op = tf.metrics.accuracy(self.labels, self.choice)
         
        one_hot_labels = tf.one_hot(indices=tf.cast(self.labels, dtype=tf.int32), depth=num_classes)     
        self.loss = tf.losses.softmax_cross_entropy(onehot_labels=one_hot_labels, logits=outputs)
         
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-2)
        self.train_operation = optimizer.minimize(loss=self.loss, global_step=tf.train.get_global_step())


In [4]:
tf.reset_default_graph()

training_steps = 2000
batch_size = 64

image_height = 28
image_width  = 28

color_channels = 1

mnist = tf.contrib.learn.datasets.load_dataset("mnist")
#extracts mnist data

train_data = mnist.train.images
train_labels = np.asarray(mnist.train.labels, dtype=np.int32) 
#preps the labels for training

cnn = CNN(image_height, image_width, color_channels, 16)

#prep the evaluation data

eval_data = mnist.train.images
eval_labels = np.asarray(mnist.train.labels, dtype=np.int32)

train_data = train_data.reshape(-1, image_height, image_width, color_channels)


dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
dataset = dataset.shuffle(buffer_size = train_labels.shape[0])
dataset =dataset.batch(batch_size)
dataset = dataset.repeat()
dataset_iterator = dataset.make_initializable_iterator()
next_element = dataset_iterator.get_next()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    sess.run(dataset_iterator.initializer)
    for step in range(training_steps):
        current_batch = sess.run(next_element)
        
        batch_inputs = current_batch[0]
        batch_labels = current_batch[1]
        sess.run((cnn.train_operation, cnn.accuracy_op), feed_dict={cnn.input_layer:batch_inputs, cnn.labels:batch_labels})
    


Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
(?, 28, 28, 1)
(?, 28, 28, 32)
(?, 14, 14, 32)
(?, 14, 14, 64)
(?, 7, 7, 64)
Flattened: (?, 3136)
Dense: (?, 1024)
Dropout: (?, 1024)
Outputs: (?, 16)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

